In [2]:
import numpy as np
import pandas as pd
from pandas import json_normalize

from bs4 import BeautifulSoup
import requests 
import time

from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from geopy.geocoders import Nominatim

import warnings
warnings.filterwarnings("ignore")

from my_functions import check_duplicates_coordinates

# Merge with prog albums

In [25]:
df_masters = pd.read_csv('Datasets/df_masters.csv')
df_masters.shape

(44783, 13)

In [63]:
df_masters[df_masters['styles'].str.contains('Prog Rock')].shape

(3572, 13)

In [21]:
df_ratings = pd.read_csv('Datasets/df_ratings.csv', keep_default_na=False)
df_ratings.shape

(352768, 5)

In [65]:
df_prog_artists = pd.read_csv('Datasets/df_prog_artists.csv')
df_prog_artists.shape

(12589, 4)

Since by default ``pd.merge`` is case-sensitive, I will create a temporary column with ``assign`` to merge the dataframes and then drop this temporary column and the one from ``df_prog_artists``, which I don't like because it's in uppercase

In [32]:
df_prog_ratings = pd.merge(
    df_ratings.assign(artist_lower=df_ratings['artist'].str.lower()), 
    df_prog_artists.assign(artist_lower=df_prog_artists['artist'].str.lower()), 
    on='artist_lower'
).drop(columns=['artist_lower', 'artist_y', 'href'])
# drop columns I don't need anymore

# rename columns
df_prog_ratings.rename(columns={'artist_x': 'artist'}, inplace=True)

# show the df
df_prog_ratings

,album_id,artist,title,rating,votes,style,country
0,41,Muse,Absolution,3.99,4411,Prog Related,United Kingdom
1,42,Muse,Showbiz,3.50,2181,Prog Related,United Kingdom
2,45,Finch,What It Is to Burn,3.69,864,Symphonic Prog,Netherlands
3,53,Dream Theater,Train of Thought,3.87,2670,Progressive Metal,United States
4,67,Black Sabbath,Paranoid,4.52,5144,Prog Related,United Kingdom
...,...,...,...,...,...,...,...
22142,516781,Buckethead,Castello Dwellers,4.50,1,Prog Related,United States
22143,516782,Buckethead,Signaling Across Time,4.00,1,Prog Related,United States
22144,516783,Buckethead,The Mines of Sigismund Fugger,4.00,1,Prog Related,United States
22145,516817,Jack O' The Clock,Portraits,4.00,1,Prog Folk,United States


In [66]:
df_prog_ratings[df_prog_ratings['votes']>10]

,album_id,artist,title,rating,votes,style,country
0,41,Muse,Absolution,3.99,4411,Prog Related,United Kingdom
1,42,Muse,Showbiz,3.50,2181,Prog Related,United Kingdom
2,45,Finch,What It Is to Burn,3.69,864,Symphonic Prog,Netherlands
3,53,Dream Theater,Train of Thought,3.87,2670,Progressive Metal,United States
4,67,Black Sabbath,Paranoid,4.52,5144,Prog Related,United Kingdom
...,...,...,...,...,...,...,...
22026,509781,Vylet Pony,Monarch of Monsters,3.77,13,Eclectic Prog,United States
22043,510543,Eidola,Mend,2.63,41,Experimental/Post Metal,United States
22046,510919,Ulver,Liminal Animals,3.26,73,Post Rock/Math rock,Norway
22137,516387,Obscura,A Sonication,2.75,48,Rock Progressivo Italiano,Italy


In [36]:
df_prog = pd.merge(df_prog_ratings, df_masters, on=['artist', 'title'])
df_prog

,album_id,artist,title,rating,votes,style,country,artist_id,master_id,main_release_id,release_country,year,album_length,tracks,release_type,genres,styles,artist_profile
0,41,Muse,Absolution,3.99,4411,Prog Related,United Kingdom,1003,11052,7781939,Europe,2003,52.18,14,"['Album', 'Stereo']",['Rock'],['Alternative Rock'],"Muse are an English rock band from Teignmouth,..."
1,42,Muse,Showbiz,3.50,2181,Prog Related,United Kingdom,1003,11001,446060,UK,1999,49.65,12,"['Album', 'Stereo']",['Rock'],"['Alternative Rock', 'Prog Rock']","Muse are an English rock band from Teignmouth,..."
2,67,Black Sabbath,Paranoid,4.52,5144,Prog Related,United Kingdom,144998,302,696772,UK,1970,0.00,8,"['LP', 'Album']",['Rock'],"['Hard Rock', 'Heavy Metal']",Considered by many to be the first heavy metal...
3,70,Yes,Relayer,4.19,1114,Symphonic Prog,United Kingdom,50263,35126,4506926,UK,1974,40.47,3,"['LP', 'Album', 'Stereo']",['Rock'],['Prog Rock'],Yes are an English rock band who achieved worl...
4,77,Iron Maiden,Dance Of Death,3.59,2074,Prog Related,United Kingdom,251595,21512,8663555,Europe,2003,67.95,11,['Album'],['Rock'],['Heavy Metal'],"English heavy metal band formed in Leyton, Eas..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1573,502885,Hats Off Gentlemen It's Adequate,Nostalgia For Infinity,3.50,1,Crossover Prog,United Kingdom,6476937,2726864,15871538,UK,2020,47.98,8,"['Album', 'Stereo']",['Rock'],['Prog Rock'],NaN
1574,502886,Hats Off Gentlemen It's Adequate,The Confidence Trick,4.00,1,Crossover Prog,United Kingdom,6476937,2726867,23954783,UK,2022,68.12,13,"['Album', 'Stereo']",['Rock'],['Prog Rock'],NaN
1575,505233,Efterklang,Things We Have In Common,3.50,2,Post Rock/Math rock,Denmark,234846,3612478,31842014,"UK, Europe & US",2024,0.00,9,"['LP', 'Album']","['Electronic', 'Rock', 'Pop']","['Abstract', 'Indie Rock', 'Folk Rock']",Experimental indie/post-rock/orchestral band f...
1576,505271,Delta Sleep,Blue Garden,3.24,23,Crossover Prog,United Kingdom,3352599,3633390,32079486,US,2024,42.80,12,"['LP', 'Album', 'Limited Edition']",['Rock'],"['Math Rock', 'Indie Rock']","Formed in 2008 as 'Sávlön' in Canterbury, UK, ..."


In [37]:
df_prog[df_prog['votes']>10]

,album_id,artist,title,rating,votes,style,country,artist_id,master_id,main_release_id,release_country,year,album_length,tracks,release_type,genres,styles,artist_profile
0,41,Muse,Absolution,3.99,4411,Prog Related,United Kingdom,1003,11052,7781939,Europe,2003,52.18,14,"['Album', 'Stereo']",['Rock'],['Alternative Rock'],"Muse are an English rock band from Teignmouth,..."
1,42,Muse,Showbiz,3.50,2181,Prog Related,United Kingdom,1003,11001,446060,UK,1999,49.65,12,"['Album', 'Stereo']",['Rock'],"['Alternative Rock', 'Prog Rock']","Muse are an English rock band from Teignmouth,..."
2,67,Black Sabbath,Paranoid,4.52,5144,Prog Related,United Kingdom,144998,302,696772,UK,1970,0.00,8,"['LP', 'Album']",['Rock'],"['Hard Rock', 'Heavy Metal']",Considered by many to be the first heavy metal...
3,70,Yes,Relayer,4.19,1114,Symphonic Prog,United Kingdom,50263,35126,4506926,UK,1974,40.47,3,"['LP', 'Album', 'Stereo']",['Rock'],['Prog Rock'],Yes are an English rock band who achieved worl...
4,77,Iron Maiden,Dance Of Death,3.59,2074,Prog Related,United Kingdom,251595,21512,8663555,Europe,2003,67.95,11,['Album'],['Rock'],['Heavy Metal'],"English heavy metal band formed in Leyton, Eas..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,484810,Plantoid,Terrapath,3.59,19,Eclectic Prog,United Kingdom,13894711,3384352,29671858,UK,2024,39.25,10,"['LP', 'Album', 'Special Edition']","['Jazz', 'Rock']","['Prog Rock', 'Psychedelic Rock']",The band began as the brainchild of Chloe Spen...
1563,486448,Ihsahn,Ihsahn,3.81,193,Tech/Extreme Prog Metal,Norway,122289,3395569,29815420,NaN,2024,0.00,11,"['WAV', 'Album', 'Stereo']",['Rock'],"['Progressive Metal', 'Avantgarde']","Composer, singer and multi-instrumentalist fro..."
1566,490484,Dirty Three,Love Changes Everything,3.32,18,Post Rock/Math rock,Australia,65791,3517705,30983092,Australia & New Zealand,2024,41.33,6,"['LP', 'Album', 'Stereo']",['Rock'],['Post Rock'],Australian instrumental rock band founded duri...
1571,495923,Kingcrow,Hopium,3.49,20,Progressive Metal,Italy,1986621,3588815,31565200,"UK, Europe & US",2024,53.33,10,['Album'],['Rock'],"['Progressive Metal', 'Prog Rock']",Kingcrow are an Italian band born in Anguillar...


In [38]:
df_prog[df_prog['votes']>10].artist.nunique()

351

# Concat with the previous ``df_master_id_0``

In [7]:
df_masters_id0_2003 = pd.read_csv('Datasets/df_masters_id0_2003.csv')
df_masters_id0_2003.shape

(850, 13)

In [15]:
df_masters_id0_2003.year.value_counts()

year
2003    850
Name: count, dtype: int64

In [16]:
df_masters_id0 = pd.read_csv('Datasets/df_masters_id0.csv')
df_masters_id0.shape

(30, 16)

In [17]:
df_masters_id0.year.value_counts()

year
2024    21
2000     9
Name: count, dtype: int64

In [18]:
df_masters_id0_concat = pd.concat([df_masters_id0_2001, df_masters_id0_2002, df_masters_id0_2003])
df_masters_id0_concat.shape

(2329, 13)

In [19]:
df_masters_id0_concat.year.value_counts()

year
2003    850
2002    778
2001    701
Name: count, dtype: int64

# Merge ``df_masters_id0`` with ``df_ratings``

In [168]:
# import the 2024 albums where master_id = 0
df_masters_id0 = pd.read_csv('Datasets/df_masters_id0_2000.csv')
df_masters_id0.shape

(703, 13)

In [22]:
# merge with the albums that have ratings
df_id0_ratings = pd.merge(df_ratings, df_masters_id0_concat, on=['artist', 'title'])
df_id0_ratings.shape

(63, 16)

In [23]:
print(f"Only {round(df_id0_ratings.shape[0] / df_masters_id0_concat.shape[0]*100, 2)} % of the albums have ratings")

Only 2.71 % of the albums have ratings


In [26]:
# some of these albums are already in df_masters
df_id0_ratings[df_id0_ratings['title'].isin(df_masters['title'].values)].sort_values('votes', ascending=False).shape[0]

35

In [28]:
# so I keep only the albums that are not in df_masters already
df_id0_ratings = df_id0_ratings[~df_id0_ratings['title'].isin(df_masters['title'].values)]
df_id0_ratings.shape[0]

28

# ``df_id0_ratings`` cleaning

## Dropping duplicated albums

In [29]:
# check if there are duplicates
df_id0_ratings.duplicated().sum()

2

In [30]:
# drop duplicates
df_id0_ratings.drop_duplicates(subset=['artist', 'title'], inplace=True)
df_id0_ratings.shape[0]

26

**Dropping live albums, compilations...**

In [31]:
# drop compilations
df_id0_ratings = df_id0_ratings[~df_id0_ratings['release_type'].str.contains('Compilation')]
df_id0_ratings.shape[0]

26

In [34]:
df_id0_ratings.sort_values(['votes', 'title', 'artist'], ascending=False)

,album_id,artist,title,rating,votes,artist_id,master_id,main_release_id,release_country,year,album_length,tracks,release_type,genres,styles,artist_profile
10,1758,Counting Crows,Hard Candy,3.35,138,262643.0,NaN,30989143,UK,2002,0.00,15,"['Album', 'Unofficial Release']",['Rock'],"['Southern Rock', 'Alternative Rock']",Counting Crows is an American rock band from B...
44,123688,The Little Explorer,The Little Explorer,3.80,27,1100295.0,NaN,2747168,UK,2003,0.00,8,['Album'],['Rock'],['Emo'],NaN
38,79784,Tygers Of Pan Tang,Mystical,2.30,10,407694.0,NaN,4632749,UK,2001,49.37,11,['Album'],['Rock'],"['Heavy Metal', 'Hard Rock']",British Heavy Metal band from Whitley Bay form...
12,6347,Howards Alias,The Chameleon Script,4.11,9,918301.0,NaN,1393346,UK,2002,37.03,11,['Album'],['Rock'],"['Ska', 'Punk', 'Indie Rock']","Band in the modern UK underground punk scene, ..."
17,17416,The String Cheese Incident,Untying The Not,3.43,7,246185.0,NaN,32414148,UK & Europe,2003,54.35,13,['Album'],"['Jazz', 'Rock', 'Folk, World, & Country']","['Bluegrass', 'Alternative Rock', 'Jazz-Rock',...",The String Cheese Incident (SCI) is an America...
28,34630,Darkflight,Under The Shadow Of Fear,3.17,6,334729.0,NaN,1301861,UK,2003,40.35,8,['Album'],['Rock'],"['Black Metal', 'Doom Metal']","Atmospheric black/doom metal duo from Varna, B..."
34,61038,Sputniks Down,Much Was Decided Before You Were Born,2.08,6,43018.0,NaN,594879,UK,2001,47.18,7,['Album'],"['Electronic', 'Rock']",['Post Rock'],"Sputniks Down were a Scottish band, made up of..."
8,1651,Miocene,Cellular Memory,3.75,6,958779.0,NaN,1809183,UK,2002,40.60,6,['Album'],['Rock'],[],"Forming in 1998, but consolidating their line-..."
52,188309,Karnataka,Delicate Flame Of Desire,3.80,5,1420825.0,NaN,1726424,UK,2003,61.50,9,['Album'],['Rock'],"['Folk Rock', 'Prog Rock']","Progressive Rock band from Swansea, Wales, UK"
35,61942,Fallen To,The Mark,2.67,3,1198317.0,NaN,3005917,UK,2002,43.80,11,['Album'],['Rock'],['Nu Metal'],NaN


In [35]:
# check one by one if necessary
try:
    df_id0_ratings.drop([49, 50, 51], axis=0, inplace=True)
    print('Rows dropped')
    print(df_id0_ratings.shape)
except:
    print("Couldn't drop these rows")
    print(df_id0_ratings.shape)

Rows dropped
(23, 16)


## Concat with the previous ``df_master_id_0``

In [36]:
df_masters_id0 = pd.read_csv('Datasets/df_masters_id0.csv')
df_masters_id0.shape

(30, 16)

In [37]:
df_masters_id0_concat = pd.concat([df_masters_id0, df_id0_ratings])
df_masters_id0_concat.shape

(53, 16)

## **Export to csv**

In [40]:
df_masters_id0_concat.to_csv('Datasets/df_masters_id0.csv', index=False)

In [3]:
df_masters_id0 = pd.read_csv('Datasets/df_masters_id0.csv')
df_masters_id0.shape

(30, 16)

In [4]:
df_masters_id0

,album_id,artist,title,rating,votes,artist_id,master_id,main_release_id,release_country,year,album_length,tracks,release_type,genres,styles,artist_profile
0,498666,Blasfeme,Black Legion,3.59,33,8287884.0,NaN,32987169,UK,2024,32.33,7,"['Album', 'Limited Edition']",['Rock'],['Black Metal'],"Blasfeme is a two piece black metal force, fue..."
1,507340,Dawnwalker,The Unknowing,3.68,11,7622357.0,NaN,32331393,UK,2024,0.00,9,"['LP', 'Album']",['Rock'],['Prog Rock'],"Post-Metal/Rock band from London, UK."
2,504016,Trash Boat,Heaven Can Wait,2.64,11,3896679.0,NaN,32921493,UK,2024,36.03,11,"['12""', '33 ⅓ RPM', 'Album', 'Stereo']",['Rock'],['Hardcore'],NaN
3,512234,Caelestra,Bastion,3.95,10,8367186.0,NaN,32682639,UK,2024,48.65,6,['Album'],['Rock'],"['Progressive Metal', 'Post-Metal', 'Black Met...","Caelestra. Bristol, UK. The creation of Fever ..."
4,496271,Other Half,Dark Ageism,3.64,5,4456943.0,NaN,31040566,UK,2024,0.00,12,"['LP', 'Album', 'Limited Edition']",['Rock'],"['Post-Hardcore', 'Punk']",NaN
5,490535,Ildganger,For Hver Tanke Mister Sj​æ​len Atter Farve,3.62,4,12767564.0,NaN,30251867,UK,2024,0.00,5,"['LP', 'Album', 'Limited Edition']",['Rock'],['Black Metal'],"Black Metal project from Aarhus, Denmark."
6,496009,Gnod,Spot Land,3.12,4,1107254.0,NaN,30624286,UK,2024,0.00,5,"['LP', 'Album', 'Limited Edition']","['Electronic', 'Rock']","['Alternative Rock', 'Industrial', 'Drone', 'M...","Gnod are a British rock band from Salford, Gre..."
7,487309,Ilat Mahru,Incipit Akkadian,3.50,3,12679967.0,NaN,30173384,UK,2024,34.33,4,"['Mini-Album', 'Limited Edition']",['Rock'],['Black Metal'],NaN
8,492912,Perseus,Into The Silence,3.50,3,220153.0,NaN,30422189,UK,2024,44.37,11,"['Album', 'Limited Edition']",['Rock'],"['Heavy Metal', 'Power Metal']",Heavy/power metal band from Italy
9,505338,Thank,I Have A Physical Body That Can Be Harmed,3.75,2,6092482.0,NaN,32117175,UK,2024,0.00,10,"['LP', 'Album']","['Hip Hop', 'Rock']","['Noise Rock', 'Post-Punk', 'No Wave']",Noise rock / post-punk / no wave band from Lee...


# ``df_masters`` cleaning

In [40]:
df_masters[df_masters[['artist', 'title']].duplicated()].shape

(57, 13)

In [89]:
df_masters[df_masters[['artist', 'title']].duplicated()]

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile
3775,407733,124122,1972400,US,Ultimate Spinach,Ultimate Spinach,1968,36.28,9,"['LP', 'Album', 'Stereo']",['Rock'],"['Psychedelic Rock', 'Garage Rock']","American Psychedelic Rock Band from Boston, Ma..."
4713,82311,6775,652106,UK,Red House Painters,Red House Painters,1993,75.62,14,['Album'],['Rock'],"['Dream Pop', 'Indie Rock', 'Slowcore']","American alternative rock group, formed in 19..."
4736,103687,774012,4856969,Europe,Dire Straits,On The Night,1993,0.00,15,"['12""', 'Stereo', 'PAL']","['Rock', 'Pop']",['Pop Rock'],British rock band formed in 1977 by Mark Knopf...
9086,2425991,201670,1891207,UK,Big Sexy Noise,Big Sexy Noise,2009,22.93,6,"['12""', 'Mini-Album', 'Limited Edition']",['Rock'],"['No Wave', 'Garage Rock', 'Avantgarde']",NaN
9117,109509,275458,2926817,UK,The Hollies,Hollies,1974,0.00,11,"['LP', 'Album']",['Rock'],"['Pop Rock', 'Soft Rock']",The Hollies are an English pop group formed in...
9275,477770,2174122,16623762,UK & Europe,Hillsong,This Is Our God,2008,0.00,17,"['DVD-Video', 'Album']","['Rock', 'Pop']","['Gospel', 'Pop Rock']",Australian Christian church organization.\r\n\...
12239,2002594,696216,3102133,US,Storm (66),Storm,1979,36.80,11,"['LP', 'Album']",['Rock'],['Classic Rock'],"American [b]rock[/b] band formed in 1978, in L..."
13717,564708,1812330,2259142,Europe,Brian Doerksen,Today,2004,74.15,16,['Album'],"['Rock', 'Pop']",['Pop Rock'],Canadian Christian singer-songwriter and wors...
14510,417060,1137938,3395996,UK & Europe,MD.45,The Craving,2004,51.68,15,"['Album', 'Remastered', 'Reissue']",['Rock'],['Heavy Metal'],MD.45 was a one-off side project that included...
14684,97325,37469,1976404,UK,Colosseum,Daughter Of Time,1970,42.48,8,"['LP', 'Album']","['Jazz', 'Rock']","['Fusion', 'Psychedelic Rock', 'Jazz-Rock', 'P...",UK band which is one of the pivotal progressiv...


In [48]:
df_masters[df_masters['title']=="Clancy"]

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile
26326,3128936,3494913,30766434,Worldwide,Twenty One Pilots,Clancy,2024,47.3,13,"['AAC', 'Album']","['Rock', 'Pop']",['Alternative Rock'],twenty one pilots (official capitalization) is...
